In [25]:
import requests
import pandas as pd

apikey= open('apikey.txt','r').readline()

def get_lat_lng(address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address},+UK&key={apikey}"
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        result = data['results'][0]
        lat = result['geometry']['location']['lat']
        lng = result['geometry']['location']['lng']
        return lat, lng
    else:
        return None, None


def get_dist_time(address,mode):
    url = f"https://maps.googleapis.com/maps/api/distancematrix/json?destinations=Heathrow+Airport+Terminal+5&origins={address}+UK&units=metric&mode={mode}&key={apikey}"
    response = requests.get(url)
    data = response.json()
    print(data)

    if data['rows'][0]['elements'][0]['status'] == 'OK' and data['status'] == 'OK':
        result = data
        dist = result['rows'][0]['elements'][0]['distance']['value']
        times = result['rows'][0]['elements'][0]['duration']['value']
        print(dist)
        return dist, times
    else:
        return None, None


In [26]:
OD=pd.read_excel('Heathrow_OD.xlsx').dropna()
OD['Local Auth'] = OD['Local Auth'].astype(str)
vals = OD['Local Auth'].astype(str)
vals

1      Aberdeen City
2      Aberdeenshire
3               Adur
4          Allerdale
5       Amber Valley
           ...      
358         Wychavon
359          Wycombe
360             Wyre
361      Wyre Forest
362             York
Name: Local Auth, Length: 362, dtype: object

In [27]:
latlist = list()
lnglist = list()
for val in vals:
     curlat, curlng = get_lat_lng(val)
     latlist.append(curlat)
     lnglist.append(curlng)


In [28]:
OD['lat'] = latlist
OD['lng'] = lnglist
OD

,Local Auth,Total Annual Demand,Mode Share Taxi,Mode Share Car,Mode Share Rail,lat,lng
1,Aberdeen City,7000.0,0.284172,0.443765,0.272063,57.149889,-2.093753
2,Aberdeenshire,1000.0,0.000000,1.000000,0.000000,57.286872,-2.381568
3,Adur,13000.0,0.000000,0.887134,0.000000,50.834817,-0.310126
4,Allerdale,3000.0,0.000000,0.218843,0.000000,54.699064,-3.199420
5,Amber Valley,13000.0,0.000000,0.857952,0.062812,53.015187,-1.482307
...,...,...,...,...,...,...,...
358,Wychavon,33000.0,0.352380,0.547394,0.000000,52.152518,-2.033095
359,Wycombe,325000.0,0.423891,0.512929,0.043526,51.638511,-0.807860
360,Wyre,3000.0,0.492094,0.000000,0.507906,53.899610,-2.809301
361,Wyre Forest,17000.0,0.138131,0.809199,0.000000,52.403805,-2.253763


In [29]:
distlist = list()
timelist = list()
for val in vals:
     curdist, curtime = get_dist_time(val,'car')
     distlist.append(curdist)
     timelist.append(curtime)

OD['Car Distance [m]'] = distlist
OD['Car Time Taken [s]'] = timelist
#OD.to_excel('heathrowflow.xlsx')
OD

{'destination_addresses': ['Terminal 5, Wallis Rd, Longford, Hounslow TW6 2GA, UK'], 'origin_addresses': ['Aberdeen, UK'], 'rows': [{'elements': [{'distance': {'text': '858 km', 'value': 857680}, 'duration': {'text': '8 hours 47 mins', 'value': 31637}, 'status': 'OK'}]}], 'status': 'OK'}
857680
{'destination_addresses': ['Terminal 5, Wallis Rd, Longford, Hounslow TW6 2GA, UK'], 'origin_addresses': ['Aberdeenshire, UK'], 'rows': [{'elements': [{'distance': {'text': '879 km', 'value': 879463}, 'duration': {'text': '9 hours 1 min', 'value': 32439}, 'status': 'OK'}]}], 'status': 'OK'}
879463
{'destination_addresses': ['Terminal 5, Wallis Rd, Longford, Hounslow TW6 2GA, UK'], 'origin_addresses': ['Adur District, UK'], 'rows': [{'elements': [{'distance': {'text': '110 km', 'value': 109867}, 'duration': {'text': '1 hour 12 mins', 'value': 4306}, 'status': 'OK'}]}], 'status': 'OK'}
109867
{'destination_addresses': ['Terminal 5, Wallis Rd, Longford, Hounslow TW6 2GA, UK'], 'origin_addresses': [

,Local Auth,Total Annual Demand,Mode Share Taxi,Mode Share Car,Mode Share Rail,lat,lng,Car Distance [m],Car Time Taken [s]
1,Aberdeen City,7000.0,0.284172,0.443765,0.272063,57.149889,-2.093753,857680.0,31637.0
2,Aberdeenshire,1000.0,0.000000,1.000000,0.000000,57.286872,-2.381568,879463.0,32439.0
3,Adur,13000.0,0.000000,0.887134,0.000000,50.834817,-0.310126,109867.0,4306.0
4,Allerdale,3000.0,0.000000,0.218843,0.000000,54.699064,-3.199420,499215.0,18571.0
5,Amber Valley,13000.0,0.000000,0.857952,0.062812,53.015187,-1.482307,217494.0,8816.0
...,...,...,...,...,...,...,...,...,...
358,Wychavon,33000.0,0.352380,0.547394,0.000000,52.152518,-2.033095,176011.0,6943.0
359,Wycombe,325000.0,0.423891,0.512929,0.043526,51.638511,-0.807860,36463.0,1950.0
360,Wyre,3000.0,0.492094,0.000000,0.507906,53.899610,-2.809301,370794.0,13998.0
361,Wyre Forest,17000.0,0.138131,0.809199,0.000000,52.403805,-2.253763,187111.0,7472.0


In [30]:
distlist = list()
timelist = list()
for val in vals:
     curdist, curtime = get_dist_time(val,'transit')
     distlist.append(curdist)
     timelist.append(curtime)

OD['Transit Distance [m]'] = distlist
OD['Transit Time Taken [s]'] = timelist
#OD.to_excel('heathrowflow.xlsx')
OD

{'destination_addresses': ['Terminal 5, Wallis Rd, Longford, Hounslow TW6 2GA, UK'], 'origin_addresses': ['Aberdeen, UK'], 'rows': [{'elements': [{'distance': {'text': '873 km', 'value': 872625}, 'duration': {'text': '8 hours 41 mins', 'value': 31242}, 'status': 'OK'}]}], 'status': 'OK'}
872625
{'destination_addresses': ['Terminal 5, Wallis Rd, Longford, Hounslow TW6 2GA, UK'], 'origin_addresses': ['Aberdeenshire, UK'], 'rows': [{'elements': [{'distance': {'text': '901 km', 'value': 900775}, 'duration': {'text': '8 hours 56 mins', 'value': 32152}, 'status': 'OK'}]}], 'status': 'OK'}
900775
{'destination_addresses': ['Terminal 5, Wallis Rd, Longford, Hounslow TW6 2GA, UK'], 'origin_addresses': ['Adur District, UK'], 'rows': [{'elements': [{'distance': {'text': '123 km', 'value': 123156}, 'duration': {'text': '2 hours 45 mins', 'value': 9889}, 'status': 'OK'}]}], 'status': 'OK'}
123156
{'destination_addresses': ['Terminal 5, Wallis Rd, Longford, Hounslow TW6 2GA, UK'], 'origin_addresses'

,Local Auth,Total Annual Demand,Mode Share Taxi,Mode Share Car,Mode Share Rail,lat,lng,Car Distance [m],Car Time Taken [s],Transit Distance [m],Transit Time Taken [s]
1,Aberdeen City,7000.0,0.284172,0.443765,0.272063,57.149889,-2.093753,857680.0,31637.0,872625.0,31242.0
2,Aberdeenshire,1000.0,0.000000,1.000000,0.000000,57.286872,-2.381568,879463.0,32439.0,900775.0,32152.0
3,Adur,13000.0,0.000000,0.887134,0.000000,50.834817,-0.310126,109867.0,4306.0,123156.0,9889.0
4,Allerdale,3000.0,0.000000,0.218843,0.000000,54.699064,-3.199420,499215.0,18571.0,532077.0,26474.0
5,Amber Valley,13000.0,0.000000,0.857952,0.062812,53.015187,-1.482307,217494.0,8816.0,251567.0,13219.0
...,...,...,...,...,...,...,...,...,...,...,...
358,Wychavon,33000.0,0.352380,0.547394,0.000000,52.152518,-2.033095,176011.0,6943.0,210714.0,12417.0
359,Wycombe,325000.0,0.423891,0.512929,0.043526,51.638511,-0.807860,36463.0,1950.0,32335.0,5204.0
360,Wyre,3000.0,0.492094,0.000000,0.507906,53.899610,-2.809301,370794.0,13998.0,429542.0,17985.0
361,Wyre Forest,17000.0,0.138131,0.809199,0.000000,52.403805,-2.253763,187111.0,7472.0,197788.0,15328.0


In [33]:
OD
OD.to_excel('heathrowflow.xlsx')